/home/rttrif/local_dev_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
llm = ChatOllama(model="gpt-oss:120b-cloud", temperature=0.0)

In [3]:
class BookInfo(BaseModel):
    title: str = Field(..., description="Название книги")
    author: str = Field(..., description="Имя автора")
    tags: list[str] = Field(..., description="Список тегов или жанров, относящихся к книге")

In [4]:
output_parser = PydanticOutputParser(pydantic_object=BookInfo)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "Название книги", "title": "Title", "type": "string"}, "author": {"description": "Имя автора", "title": "Author", "type": "string"}, "tags": {"description": "Список тегов или жанров, относящихся к книге", "items": {"type": "string"}, "title": "Tags", "type": "array"}}, "required": ["title", "author", "tags"]}\n```'

In [5]:
prompt = PromptTemplate(
    template="Ответь на вопрос в требуемом формате.\n{format_instructions}\nВопрос: {user_question}\nОтвет:",
    input_variables=["user_question"],
    partial_variables={"format_instructions": format_instructions}
)

In [7]:
chain = prompt | llm
result = chain.invoke({"user_question": "Расскажи о книге '1984' Джорджа Оруэлла."})
result.content

'{\n  "title": "1984",\n  "author": "Джордж Оруэлл",\n  "tags": [\n    "антиутопия",\n    "политика",\n    "дистопия",\n    "классика"\n  ]\n}'